In [2]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from subprocess import check_output
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import os
import gc
import time
import re
import requests
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

df=pd.read_excel('input.xlsx')

In [3]:
df.head()

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...


#### Using Beautiful Soup to scrape article text from the article link

In [10]:
for i,j in df.iterrows():
    filename=str(j['URL_ID'])+".txt"
    print(j['URL_ID'])
    print(j['URL'])
    headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
    page=requests.get(j['URL'],headers=headers,)
    soup=BeautifulSoup(page.content,'html.parser')
    pattern="<[^\>]*>"
    title = re.sub(pattern,"",str(soup.find('title') ))
    content=re.sub(pattern,"",str(soup.find_all('p')))
    with open (filename, 'w', newline='',encoding='utf-8') as f:
        f.write(title)
        f.write("\n")
        f.write(content)
        f.write("\n")
    time.sleep(5)

37
https://insights.blackcoffer.com/ai-in-healthcare-to-improve-patient-outcomes/
38
https://insights.blackcoffer.com/what-if-the-creation-is-taking-over-the-creator/
39
https://insights.blackcoffer.com/what-jobs-will-robots-take-from-humans-in-the-future/


KeyboardInterrupt: 

#### Reading stop word files

In [4]:
from nltk.tokenize import sent_tokenize, word_tokenize

auditor= open("StopWords\\StopWords_Auditor.txt",'r').read()
currency=open("StopWords\\StopWords_Currencies.txt",'r').read()
datesandnumbers=open("StopWords\\StopWords_DatesandNumbers.txt",'r').read()
generic=open("StopWords\\StopWords_Generic.txt",'r').read()
genericlong=open("StopWords\\StopWords_GenericLong.txt",'r').read()
geographic=open("StopWords\\StopWords_Geographic.txt",'r').read()
names=open("StopWords\\StopWords_Names.txt",'r').read()


STOP_WORDS=auditor+currency+datesandnumbers+generic+genericlong+geographic+names



In [5]:
print(STOP_WORDS[:100])

ERNST
YOUNG
DELOITTE
TOUCHE
KPMG
PRICEWATERHOUSECOOPERS
PRICEWATERHOUSE
COOPERS
AFGHANI  | Afghanist


In [6]:
df.head()

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...


#### Reading article text from the scaped text files into the output dataframe

In [7]:
ls

 Volume in drive C has no label.
 Volume Serial Number is FE7D-6C16

 Directory of C:\Users\tomar\Downloads\Case Studies\20211030 Test Assignment-20221116T043615Z-001\20211030 Test Assignment Solution

21-11-2022  03:47 PM    <DIR>          .
21-11-2022  03:47 PM    <DIR>          ..
21-11-2022  03:47 PM    <DIR>          .ipynb_checkpoints
16-11-2022  10:07 AM            14,625 Input.xlsx
16-11-2022  10:07 AM    <DIR>          MasterDictionary
16-11-2022  10:07 AM            11,952 Objective.docx
16-11-2022  10:07 AM            15,877 Output Data Structure.xlsx
21-11-2022  03:36 PM    <DIR>          Scraped Text
21-11-2022  03:37 PM           120,298 SentimentalAnalysisAssignmentSolution (Gaurav Tomar).ipynb
16-11-2022  10:07 AM    <DIR>          StopWords
16-11-2022  10:07 AM            14,551 Text Analysis.docx
               5 File(s)        177,303 bytes
               6 Dir(s)   3,278,475,264 bytes free


In [10]:
output_df=df[['URL_ID', 'URL']].copy()
text_list = list()

for index,row in output_df.iterrows():
    article_text=(open('Scraped Text\\{}.txt'.format(str(row["URL_ID"])), encoding='utf-8').read())
    text_list.append(article_text)
output_df['text']=text_list

In [11]:
output_df.iloc[0][0]

37

In [12]:
output_df.text.fillna("", inplace=True)

In [13]:
output_df.head()

,URL_ID,URL,text
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes -...
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creato...
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...
3,40,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...
4,41,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us? - Blackcof...


In [14]:
output_df.tail()

,URL_ID,URL,text
109,146,https://insights.blackcoffer.com/blockchain-fo...,Blockchain for Payments - Blackcoffer Insights...
110,147,https://insights.blackcoffer.com/the-future-of...,The future of Investing - Blackcoffer Insights...
111,148,https://insights.blackcoffer.com/big-data-anal...,Big Data Analytics in Healthcare - Blackcoffer...
112,149,https://insights.blackcoffer.com/business-anal...,Business Analytics In The Healthcare Industry ...
113,150,https://insights.blackcoffer.com/challenges-an...,Challenges and Opportunities of Big Data in He...


#### Removing the stopwords (given) from the article text

In [18]:
stopwords_list=list(STOP_WORDS.split("\n"))
from nltk import word_tokenize
article_word_list = list()
for article_text in output_df['text']:
    token=word_tokenize(article_text)
    article_words = [word for word in token if word not in stopwords_list]
    article_word_list.append(article_words)

In [19]:
print(article_word_list[0])

['healthcare', 'Improve', 'Patient', 'Outcomes', '-', 'Blackcoffer', 'Insights', '[', 'Introduction', ',', '“', 'If', 'kills', '10', 'million', 'people', 'decades', ',', 'a', 'highly', 'infectious', 'virus', 'a', 'war', '.', 'Not', 'missiles', 'microbes.', '”', 'Bill', 'Gates', '’', 'remarks', 'a', 'conference', '2014', ',', 'world', 'avoided', 'Ebola', 'outbreak', '.', 'When', ',', 'unprecedented', ',', 'invisible', 'virus', 'hit', ',', 'met', 'overwhelmed', 'unprepared', 'healthcare', 'system', 'oblivious', 'population', '.', 'This', 'public', 'health', 'emergency', 'demonstrated', 'lack', 'scientific', 'consideration', 'underlined', 'alarming', 'robust', 'innovations', 'health', 'medical', 'facilities', '.', 'For', 'past', 'years', ',', 'artificial', 'intelligence', 'proven', 'tangible', 'potential', 'healthcare', 'sectors', ',', 'clinical', 'practices', ',', 'translational', 'medical', 'biomedical', 'research.', ',', 'After', 'case', 'detected', 'China', 'December', '31st', '2019',

In [21]:
positivefile=open("MasterDictionary\\positive-words.txt",encoding='ISO-8859-1').read()
positive_list=list(positivefile.split("\n"))
negativefile=open("MasterDictionary\\negative-words.txt",encoding='ISO-8859-1').read()
negative_list=list(negativefile.split("\n"))

In [22]:
article_word_list[0]

['healthcare',
 'Improve',
 'Patient',
 'Outcomes',
 '-',
 'Blackcoffer',
 'Insights',
 '[',
 'Introduction',
 ',',
 '“',
 'If',
 'kills',
 '10',
 'million',
 'people',
 'decades',
 ',',
 'a',
 'highly',
 'infectious',
 'virus',
 'a',
 'war',
 '.',
 'Not',
 'missiles',
 'microbes.',
 '”',
 'Bill',
 'Gates',
 '’',
 'remarks',
 'a',
 'conference',
 '2014',
 ',',
 'world',
 'avoided',
 'Ebola',
 'outbreak',
 '.',
 'When',
 ',',
 'unprecedented',
 ',',
 'invisible',
 'virus',
 'hit',
 ',',
 'met',
 'overwhelmed',
 'unprepared',
 'healthcare',
 'system',
 'oblivious',
 'population',
 '.',
 'This',
 'public',
 'health',
 'emergency',
 'demonstrated',
 'lack',
 'scientific',
 'consideration',
 'underlined',
 'alarming',
 'robust',
 'innovations',
 'health',
 'medical',
 'facilities',
 '.',
 'For',
 'past',
 'years',
 ',',
 'artificial',
 'intelligence',
 'proven',
 'tangible',
 'potential',
 'healthcare',
 'sectors',
 ',',
 'clinical',
 'practices',
 ',',
 'translational',
 'medical',
 'biome

#### Calculating Sentiment Analysis using positive and negative words

In [23]:
positive_score_list = list()
negative_score_list = list()
polarity_list=list()
subjectivity_list=list()
for article_words in article_word_list:
    positive_score=len([words for words in article_words if words in list(positive_list)])
    negative_score=len([words for words in article_words if words in list(negative_list)])
    polarity=((positive_score-negative_score)/((positive_score+negative_score)+0.000001))
    subjectivity=((positive_score+negative_score)/((len(article_words))+0.000001))
    positive_score_list.append(positive_score)
    negative_score_list.append(negative_score)
    polarity_list.append(polarity)
    subjectivity_list.append(subjectivity)
output_df["POSITIVE SCORE"]=positive_score_list
output_df["NEGATIVE SCORE"]=negative_score_list
output_df["POLARITY SCORE"]=polarity_list
output_df["SUBJECTIVITY SCORE"]=subjectivity_list

In [24]:
output_df.head()

,URL_ID,URL,text,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes -...,63,31,0.340426,0.072812
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creato...,55,36,0.208791,0.100552
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,65,34,0.313131,0.084615
3,40,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,55,21,0.447368,0.079415
4,41,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us? - Blackcof...,48,22,0.371429,0.060606


In [25]:
stop_words_nltk=stopwords.words('english')
stop_words_nltk[:5]

['i', 'me', 'my', 'myself', 'we']

In [26]:
def remove_punctuations(article_text):
    return re.sub(r'[^\w]', ' ', article_text)

In [27]:
output_df["processed_article_text"]=output_df["text"].apply(remove_punctuations)

#### Functions for removing nltk stopwords and counting personal pronouns

In [28]:
def clean_words(article_text):
    cleanword=article_text.split()
    clean_words_list = [word for word in cleanword if word not in stop_words_nltk]
    return clean_words_list
def clean_word_count(article_text):
    return len(clean_words(article_text))
def personal_pronouns(article_text):
    pattern=re.compile(r'\b(I|we|my|ours|(?-i:us))\b')
    personal_pronouns_list=(re.findall(pattern, article_text))
    return personal_pronouns_list
def personal_pronouns_count(article_text):
    return len(personal_pronouns(article_text))

#### Testing the above functions

In [29]:
w="hello my name is gaurav tomar, this is a stop word us US transparency"
clean_word_list = clean_words(w)
personal_pronouns_list = personal_pronouns(w)
print(clean_word_list, len(clean_word_list))
print(personal_pronouns_list, len(personal_pronouns_list))

['hello', 'name', 'gaurav', 'tomar,', 'stop', 'word', 'us', 'US', 'transparency'] 9
['my', 'us'] 2


#### Applying these functions to the output dataframe

In [30]:
output_df["clean_words"]=output_df["processed_article_text"].apply(clean_words)
output_df["personal_pronouns_text"]=output_df["processed_article_text"].apply(personal_pronouns)
output_df["WORD COUNT"]=output_df["processed_article_text"].apply(clean_word_count)
output_df["PERSONAL PRONOUNS"]=output_df["processed_article_text"].apply(personal_pronouns_count)

In [31]:
output_df.head(1)

,URL_ID,URL,text,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,processed_article_text,clean_words,personal_pronouns_text,WORD COUNT,PERSONAL PRONOUNS
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes -...,63,31,0.340426,0.072812,AI in healthcare to Improve Patient Outcomes ...,"[AI, healthcare, Improve, Patient, Outcomes, B...",[us],1203,1


In [32]:
output_df.head()

,URL_ID,URL,text,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,processed_article_text,clean_words,personal_pronouns_text,WORD COUNT,PERSONAL PRONOUNS
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes -...,63,31,0.340426,0.072812,AI in healthcare to Improve Patient Outcomes ...,"[AI, healthcare, Improve, Patient, Outcomes, B...",[us],1203,1
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creato...,55,36,0.208791,0.100552,What if the Creation is Taking Over the Creato...,"[What, Creation, Taking, Over, Creator, Blackc...","[we, us, us, us, we, we]",809,6
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,65,34,0.313131,0.084615,What Jobs Will Robots Take From Humans in The ...,"[What, Jobs, Will, Robots, Take, From, Humans,...","[us, us]",1051,2
3,40,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,55,21,0.447368,0.079415,Will Machine Replace The Human in the Future o...,"[Will, Machine, Replace, The, Human, Future, W...","[us, we, us, we, we, we, we, we, we, we, we, w...",943,17
4,41,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us? - Blackcof...,48,22,0.371429,0.060606,Will AI Replace Us or Work With Us Blackcof...,"[Will, AI, Replace, Us, Work, With, Us, Blackc...","[us, we, we, we, we, us, us, we, we, us, us, us]",1034,12


In [33]:
def avg_word_length(article_text):
    article_words = article_text.split()
    average_word_length = ((sum(len(word) for word in article_words)) / len(article_words))
    return average_word_length

In [34]:
print(avg_word_length(w))

4.0


In [37]:
output_df["AVG WORD LENGTH"]=output_df["processed_article_text"].fillna("").apply(avg_word_length)

In [38]:
output_df.head(1)

,URL_ID,URL,text,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,processed_article_text,clean_words,personal_pronouns_text,WORD COUNT,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes -...,63,31,0.340426,0.072812,AI in healthcare to Improve Patient Outcomes ...,"[AI, healthcare, Improve, Patient, Outcomes, B...",[us],1203,1,5.556962


#### Function to count the syllabels in a given word, this code is not mine and it is taken from: {}

In [39]:

def sylco(article_text) :
    word = article_text.lower()

    # exception_add are words that need extra syllables
    # exception_del are words that need less syllables

    exception_add = ['serious','crucial']
    exception_del = ['fortunately','unfortunately']

    co_one = ['cool','coach','coat','coal','count','coin','coarse','coup','coif','cook','coign','coiffe','coof','court']
    co_two = ['coapt','coed','coinci']

    pre_one = ['preach']

    syls = 0 #added syllable number
    disc = 0 #discarded syllable number

    #1) if letters < 3 : return 1
    if len(word) <= 3 :
        syls = 1
        return syls

    #2) if doesn't end with "ted" or "tes" or "ses" or "ied" or "ies", discard "es" and "ed" at the end.
    # if it has only 1 vowel or 1 set of consecutive vowels, discard. (like "speed", "fled" etc.)

    if word[-2:] == "es" or word[-2:] == "ed" :
        doubleAndtripple_1 = len(re.findall(r'[eaoui][eaoui]',word))
        if doubleAndtripple_1 > 1 or len(re.findall(r'[eaoui][^eaoui]',word)) > 1 :
            if word[-3:] == "ted" or word[-3:] == "tes" or word[-3:] == "ses" or word[-3:] == "ied" or word[-3:] == "ies" :
                pass
            else :
                disc+=1

    #3) discard trailing "e", except where ending is "le"  

    le_except = ['whole','mobile','pole','male','female','hale','pale','tale','sale','aisle','whale','while']

    if word[-1:] == "e" :
        if word[-2:] == "le" and word not in le_except :
            pass

        else :
            disc+=1

    #4) check if consecutive vowels exists, triplets or pairs, count them as one.

    doubleAndtripple = len(re.findall(r'[eaoui][eaoui]',word))
    tripple = len(re.findall(r'[eaoui][eaoui][eaoui]',word))
    disc+=doubleAndtripple + tripple

    #5) count remaining vowels in word.
    numVowels = len(re.findall(r'[eaoui]',word))

    #6) add one if starts with "mc"
    if word[:2] == "mc" :
        syls+=1

    #7) add one if ends with "y" but is not surrouned by vowel
    if word[-1:] == "y" and word[-2] not in "aeoui" :
        syls +=1

    #8) add one if "y" is surrounded by non-vowels and is not in the last word.

    for i,j in enumerate(word) :
        if j == "y" :
            if (i != 0) and (i != len(word)-1) :
                if word[i-1] not in "aeoui" and word[i+1] not in "aeoui" :
                    syls+=1

    #9) if starts with "tri-" or "bi-" and is followed by a vowel, add one.

    if word[:3] == "tri" and word[3] in "aeoui" :
        syls+=1

    if word[:2] == "bi" and word[2] in "aeoui" :
        syls+=1

    #10) if ends with "-ian", should be counted as two syllables, except for "-tian" and "-cian"

    if word[-3:] == "ian" : 
    #and (word[-4:] != "cian" or word[-4:] != "tian") :
        if word[-4:] == "cian" or word[-4:] == "tian" :
            pass
        else :
            syls+=1

    #11) if starts with "co-" and is followed by a vowel, check if exists in the double syllable dictionary, if not, check if in single dictionary and act accordingly.

    if word[:2] == "co" and word[2] in 'eaoui' :

        if word[:4] in co_two or word[:5] in co_two or word[:6] in co_two :
            syls+=1
        elif word[:4] in co_one or word[:5] in co_one or word[:6] in co_one :
            pass
        else :
            syls+=1

    #12) if starts with "pre-" and is followed by a vowel, check if exists in the double syllable dictionary, if not, check if in single dictionary and act accordingly.

    if word[:3] == "pre" and word[3] in 'eaoui' :
        if word[:6] in pre_one :
            pass
        else :
            syls+=1

    #13) check for "-n't" and cross match with dictionary to add syllable.

    negative = ["doesn't", "isn't", "shouldn't", "couldn't","wouldn't"]

    if word[-3:] == "n't" :
        if word in negative :
            syls+=1
        else :
            pass   

    #14) Handling the exceptional words.

    if word in exception_del :
        disc+=1

    if word in exception_add :
        syls+=1     

    # calculate the output
    syllable_count=numVowels - disc + syls
    return syllable_count


#### Function to find number of syllables per word (as the explanation was not clear so i calculated average syllables)

In [40]:
def avg_syl_per_word(article_text):
    word_syll_map = dict()
    for word  in article_text.split():
        word_syll_map[word] = sylco(word)
    avg_syl = sum(list(word_syll_map.values()))/len(list(word_syll_map.keys()))
    return avg_syl
output_df["SYLLABLE PER WORD"]=output_df["processed_article_text"].apply(avg_syl_per_word)


#### Function to find number of Complex Words

In [41]:
def find_complex_words(article_text):
    complex_word_count = [word for word in article_text.split() if sylco(word)>2]
    return complex_word_count
def number_of_complex_words(article_text):
    return len(find_complex_words(article_text))

#### Adding the complex words and number of complex words to the output dataframe

In [42]:
output_df["complex_words"]=output_df["processed_article_text"].apply(find_complex_words)
output_df["COMPLEX WORD COUNT"]=output_df["processed_article_text"].apply(number_of_complex_words)

In [43]:
output_df.head()

,URL_ID,URL,text,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,processed_article_text,clean_words,personal_pronouns_text,WORD COUNT,PERSONAL PRONOUNS,AVG WORD LENGTH,SYLLABLE PER WORD,complex_words,COMPLEX WORD COUNT
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes -...,63,31,0.340426,0.072812,AI in healthcare to Improve Patient Outcomes ...,"[AI, healthcare, Improve, Patient, Outcomes, B...",[us],1203,1,5.556962,2.275449,"[Blackcoffer, Introduction, anything, infectio...",413
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creato...,55,36,0.208791,0.100552,What if the Creation is Taking Over the Creato...,"[What, Creation, Taking, Over, Creator, Blackc...","[we, us, us, us, we, we]",809,6,4.718232,1.946309,"[Blackcoffer, fascination, potential, tinkerin...",196
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,65,34,0.313131,0.084615,What Jobs Will Robots Take From Humans in The ...,"[What, Jobs, Will, Robots, Take, From, Humans,...","[us, us]",1051,2,5.351841,2.200546,"[Blackcoffer, Introduction, rapidly, evolving,...",365
3,40,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,55,21,0.447368,0.079415,Will Machine Replace The Human in the Future o...,"[Will, Machine, Replace, The, Human, Future, W...","[us, we, us, we, we, we, we, we, we, we, we, w...",943,17,4.780822,1.952456,"[Blackcoffer, Anything, intelligence, Artifici...",243
4,41,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us? - Blackcof...,48,22,0.371429,0.060606,Will AI Replace Us or Work With Us Blackcof...,"[Will, AI, Replace, Us, Work, With, Us, Blackc...","[us, we, we, we, we, us, us, we, we, us, us, us]",1034,12,4.936999,2.007742,"[Blackcoffer, intelligence, invention, humanit...",295


#### Calculating the Average Sentence Length , Percentage of Complex Words , FOG index and adding them to the output dataframe

In [44]:
import nltk
def avg_sent_length(article_text):
    average_sent_length=len(word_tokenize(article_text))/len(nltk.sent_tokenize(article_text))
    return average_sent_length
def prcnt_complex_words(article_text):
    percentage_complex_words=number_of_complex_words(article_text)/len(word_tokenize(article_text))
    return percentage_complex_words

def fog(article_text):
    Fog_index = 0.4 * (avg_sent_length(article_text) + prcnt_complex_words(article_text))
    return Fog_index
output_df["AVG SENTENCE LENGTH"]=output_df["text"].apply(avg_sent_length)
output_df["PERCENTAGE OF COMPLEX WORDS"]=output_df["text"].apply(prcnt_complex_words)
output_df["FOG INDEX"]=output_df["text"].apply(fog)

In [45]:
output_df["AVG NUMBER OF WORDS PER SENTENCE"]=output_df["AVG SENTENCE LENGTH"].copy()
output_df.head()

,URL_ID,URL,text,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,processed_article_text,clean_words,personal_pronouns_text,WORD COUNT,PERSONAL PRONOUNS,AVG WORD LENGTH,SYLLABLE PER WORD,complex_words,COMPLEX WORD COUNT,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes -...,63,31,0.340426,0.072812,AI in healthcare to Improve Patient Outcomes ...,"[AI, healthcare, Improve, Patient, Outcomes, B...",[us],1203,1,5.556962,2.275449,"[Blackcoffer, Introduction, anything, infectio...",413,38.711538,0.217586,15.571650,38.711538
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creato...,55,36,0.208791,0.100552,What if the Creation is Taking Over the Creato...,"[What, Creation, Taking, Over, Creator, Blackc...","[we, us, us, us, we, we]",809,6,4.718232,1.946309,"[Blackcoffer, fascination, potential, tinkerin...",196,23.956522,0.131881,9.635361,23.956522
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,65,34,0.313131,0.084615,What Jobs Will Robots Take From Humans in The ...,"[What, Jobs, Will, Robots, Take, From, Humans,...","[us, us]",1051,2,5.351841,2.200546,"[Blackcoffer, Introduction, rapidly, evolving,...",365,29.106061,0.201458,11.723007,29.106061
3,40,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,55,21,0.447368,0.079415,Will Machine Replace The Human in the Future o...,"[Will, Machine, Replace, The, Human, Future, W...","[us, we, us, we, we, we, we, we, we, we, we, w...",943,17,4.780822,1.952456,"[Blackcoffer, Anything, intelligence, Artifici...",243,23.592105,0.139989,9.492838,23.592105
4,41,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us? - Blackcof...,48,22,0.371429,0.060606,Will AI Replace Us or Work With Us Blackcof...,"[Will, AI, Replace, Us, Work, With, Us, Blackc...","[us, we, we, we, we, us, us, we, we, us, us, us]",1034,12,4.936999,2.007742,"[Blackcoffer, intelligence, invention, humanit...",295,29.194030,0.158487,11.741007,29.194030


In [46]:
list_of_required_columns=["URL_ID","URL","POSITIVE SCORE","NEGATIVE SCORE","POLARITY SCORE","SUBJECTIVITY SCORE","AVG SENTENCE LENGTH","PERCENTAGE OF COMPLEX WORDS","FOG INDEX","AVG NUMBER OF WORDS PER SENTENCE","COMPLEX WORD COUNT","WORD COUNT","SYLLABLE PER WORD","PERSONAL PRONOUNS","AVG WORD LENGTH"]

In [47]:
final_df = output_df[list_of_required_columns]

#### As there was no clear explanation about AVG SENTENCE LENGTH and AVG NUMBER OF WORDS PER SENTENCE in the text analysis document and they seemed to be same so calculated them by using the same formula

In [48]:
final_df.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,63,31,0.340426,0.072812,38.711538,0.217586,15.571650,38.711538,413,1203,2.275449,1,5.556962
1,38,https://insights.blackcoffer.com/what-if-the-c...,55,36,0.208791,0.100552,23.956522,0.131881,9.635361,23.956522,196,809,1.946309,6,4.718232
2,39,https://insights.blackcoffer.com/what-jobs-wil...,65,34,0.313131,0.084615,29.106061,0.201458,11.723007,29.106061,365,1051,2.200546,2,5.351841
3,40,https://insights.blackcoffer.com/will-machine-...,55,21,0.447368,0.079415,23.592105,0.139989,9.492838,23.592105,243,943,1.952456,17,4.780822
4,41,https://insights.blackcoffer.com/will-ai-repla...,48,22,0.371429,0.060606,29.194030,0.158487,11.741007,29.194030,295,1034,2.007742,12,4.936999


In [49]:
final_df.to_csv('output.csv', index=False)